In [2]:
search_for = 132
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003945827484130859
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 1268606955
type: [1, 1, 1, 1, 132] 132
cases of this type: 303595776
100000 56.83138580581486
200000 100.25006163563812
300000 100.99243457817784
400000 99.56553839314684
500000 99.07960105134805
600000 103.0331288825373
700000 100.02221358386407
800000 97.56681290666235
900000 97.59654350462196
1000000 99.33548018053334
1100000 97.10907216009579
1200000 26.067495699548484
1300000 97.27523626424913
1400000 97.31983497717891
1500000 96.93442710777065
1600000 96.80894733035375
1700000 100.33819544817673
1800000 102.90479281906725
1900000 97.41124891943889
2000000 97.33522264952018
2100000 97.20745394998043
2200000 99.26213949253555
2300000 75.86150692101566
2400000 46.25933332880033
2500000 92.99541969763895
2600000 92.39128587654638
2700000 92.86825389268601
2800000 91.8753786292786
2900000 95.93630497290383
3000000 92.883326590

29600000 101.62815150021996
29700000 68.19894984311435
29800000 76.45631227111612
29900000 106.94460837336243
30000000 100.45752442495753
30100000 99.72249424461897
30200000 41.40041874183385
30300000 100.47476937356826
30400000 101.38402618064748
30500000 99.18277070395304
30600000 95.42809085861933
30700000 99.0230969984798
30800000 95.07811927917024
30900000 95.29727640060207
31000000 90.11176418400422
31100000 26.131474388120974
31200000 97.71021059710475
31300000 93.45401948954529
31400000 93.28672705337928
31500000 97.2240418366783
31600000 97.65137237470374
31700000 97.91811269584079
31800000 99.73293567311633
31900000 97.5773190020077
32000000 55.79394769463225
32100000 86.50500963573988
32200000 102.93681241643748
32300000 103.86299762341837
32400000 106.3159334501944
32500000 33.613485798592635
32600000 108.38883764393644
32700000 105.5477347792671
32800000 103.25497701564817
32900000 104.6950808318923
33000000 102.72786925462303
33100000 98.81700472148344
33200000 98.5613532

59900000 101.41403809111301
60000000 101.12828002455521
60100000 103.45942918525745
60200000 103.24569469182484
60300000 36.82528211815452
60400000 98.9243190068619
60500000 95.65176742023334
60600000 96.24259422369832
60700000 93.08575308170873
60800000 89.4808083969107
60900000 90.75721774537614
61000000 23.592176357884938
61100000 95.9560562111568
61200000 90.46260734931195
61300000 90.89436161458325
61400000 89.64021169624571
61500000 96.43448693519086
61600000 97.58815883307837
61700000 37.77447280952451
61800000 105.21236898689462
61900000 102.3948287824731
62000000 99.92512631158019
62100000 105.98472247721337
62200000 97.46181458307805
62300000 97.21706972245855
62400000 100.82307708330262
62500000 97.71012469416874
62600000 50.1698320575243
62700000 81.13634988670944
62800000 92.59672905466007
62900000 91.30574658986885
63000000 84.30900271924232
63100000 86.58537455106367
63200000 88.23199633126546
63300000 23.13372092359689
63400000 91.99190179981939
63500000 88.096878731698

90300000 77.13765560217584
90400000 47.11731769935297
90500000 61.69168115908506
90600000 74.5915052907169
90700000 80.34590609313737
90800000 76.89438899836871
90900000 19.330591728216273
91000000 80.43901789603504
91100000 75.86452608763162
91200000 74.45526508395267
91300000 71.6567787204115
91400000 41.77124604664586
91500000 78.68937303782218
91600000 84.94512349586554
91700000 82.91760708968592
91800000 88.32594882702956
91900000 87.53688184730173
92000000 86.85586254086819
92100000 83.62790474143316
92200000 86.03105311770112
92300000 89.804995558356
92400000 85.01992750690536
92500000 80.40393238655435
92600000 79.36053882616038
92700000 60.16337362049472
92800000 49.74471303012397
92900000 75.13832215761676
93000000 79.81716679390789
93100000 75.37041442909478
93200000 20.44029677382099
93300000 83.8273458452508
93400000 74.73394530617635
93500000 78.26695476332581
93600000 63.959650939300154
93700000 49.59463878666129
93800000 74.32828305640805
93900000 82.65795976616701
9400

119900000 53.985687318883535
120000000 40.88507661747956
120100000 51.255344300962285
120200000 53.57774067898132
120300000 41.47946348552662
120400000 44.34144974438889
120500000 33.86596281124555
120600000 25.03844963303168
120700000 54.29922575324947
120800000 10.028995019780183
120900000 39.99259432143661
121000000 29.778077094694712
121100000 28.89932684067501
121200000 38.496613009595954
121300000 40.39910267279811
121400000 41.820332568865126
121500000 43.755904497302296
121600000 40.59076693424927
121700000 49.00949016247325
121800000 44.766299577630015
121900000 44.232788004472795
122000000 48.770347106476535
122100000 42.08896284480226
122200000 45.03789646832256
122300000 38.088392025561426
122400000 44.21887178202407
122500000 38.01408609027878
122600000 49.1193031721869
122700000 37.48442938173309
122800000 34.1859203113622
122900000 20.524128175256905
123000000 36.774074281951876
123100000 11.276837568813491
123200000 41.19659440850116
123300000 23.281748038695728
1234000

...

1190100000 3.6629739723398385
1190200000 3.285350670851148
1190300000 3.2388361885558097
1190400000 3.637664425613046
1190500000 3.6754997866132624
1190600000 3.0515405298039315
1190700000 3.942552329102208
1190800000 3.1789537996849355
1190900000 2.673569672237115
1191000000 3.1307094646741747
1191100000 3.76464311788199
1191200000 3.1087172169944015
1191300000 3.6843515681183936
1191400000 3.489322877774358
1191500000 3.3404864529799885
1191600000 4.549539540914754
best so far: 0
type: [1, 4, 1, 3, 11] 132
cases of this type: 1581228
1191700000 2.5147180395010778
1191800000 1.5367947379824123
1191900000 1.0958894234479069
1192000000 1.477836861690359
1192100000 1.6264316572273405
1192200000 1.2343277309187153
1192300000 1.5897737116561332
1192400000 2.199068721555372
1192500000 1.6185897603386439
1192600000 1.379405758897129
1192700000 1.7962522018677491
1192800000 1.665954282669425
1192900000 1.3008817216072248
1193000000 1.5038673320282268
1193100000 1.5495015035667616
1193200000 1

1203900000 4.064963043595145
1204000000 3.6139960100779134
1204100000 2.8712570896707943
1204200000 4.27610048993962
1204300000 3.442717329719027
1204400000 2.8596213326566056
1204500000 4.3447252720978256
1204600000 3.511622723061681
1204700000 2.9247516479872133
1204800000 4.524172052033455
1204900000 3.091802641010198
1205000000 3.7082957402260304
1205100000 4.214735232865383
1205200000 2.663902886188507
1205300000 3.7110939024065766
1205400000 4.155297757851809
1205500000 2.4546933641331163
1205600000 3.245032639995572
1205700000 4.020666763538083
1205800000 2.503832810861968
1205900000 3.5481677696241736
1206000000 4.107868425022483
1206100000 2.313376535554783
1206200000 3.9394681145099866
1206300000 3.9172230635180174
1206400000 2.357762748455392
1206500000 4.019990936733031
1206600000 3.7935864621183275
1206700000 2.895070035686238
1206800000 4.235614739053041
1206900000 3.7332484592677653
1207000000 3.140784482914011
1207100000 2.318968791275415
1207200000 0.5631288068800867
1

1231100000 0.6183358976878226
1231200000 1.0283341602823635
1231300000 1.0874105908311373
1231400000 0.7402431106040214
1231500000 1.1358234619462193
1231600000 1.0666836020494301
1231700000 0.7052259935268661
1231800000 1.049811019308279
1231900000 0.9851168492748638
1232000000 0.7396674438826408
1232100000 1.1287203474338752
1232200000 0.9872373210372892
1232300000 0.8093944616303146
1232400000 1.0397521037491562
1232500000 0.8642793946361377
1232600000 0.9079517330921358
1232700000 1.0648526245268881
1232800000 0.9035145189083749
1232900000 0.9119388479259577
best so far: 0
type: [2, 1, 1, 6, 11] 132
cases of this type: 3162456
1233000000 0.9301424167799055
1233100000 0.5927399628459314
1233200000 0.8864433154627748
1233300000 1.0055918125722707
1233400000 0.5427233344337311
1233500000 0.8860867363447712
1233600000 0.8657015104125142
1233700000 0.5011904521118205
1233800000 0.8928775292226871
1233900000 0.8286529180217881
1234000000 0.5909070025880204
1234100000 0.9560110922034251
1

1252600000 0.4671090925477644
1252700000 0.40652186192343964
1252800000 0.4449818656798303
1252900000 0.45585429403560024
1253000000 0.42801399770048265
1253100000 0.417237909279555
1253200000 0.4757237753198379
best so far: 0
type: [2, 2, 1, 3, 11] 132
cases of this type: 790614
1253300000 0.31059330137389235
1253400000 0.24176569767619174
1253500000 0.2512570738094018
1253600000 0.23309330547785095
1253700000 0.2625796999994913
1253800000 0.2297029439123339
1253900000 0.25193288674273756
1254000000 0.19317475157228112
best so far: 0
type: [2, 2, 1, 11, 3] 132
cases of this type: 215622
1254100000 0.2454945090068645
1254200000 0.2631874621618224
best so far: 0
type: [2, 2, 1, 33, 1] 132
cases of this type: 71874
1254300000 0.15623906550807753
best so far: 0
type: [2, 2, 3, 1, 11] 132
cases of this type: 263538
1254400000 0.21137669577987658
1254500000 0.1174396988528106
best so far: 0
type: [2, 2, 3, 11, 1] 132
cases of this type: 23958
best so far: 0
type: [2, 2, 11, 1, 3] 132
cases 

best so far: 0
type: [3, 2, 22, 1, 1] 132
cases of this type: 968
best so far: 0
type: [3, 4, 1, 1, 11] 132
cases of this type: 58564
best so far: 0
type: [3, 4, 1, 11, 1] 132
cases of this type: 5324
best so far: 0
type: [3, 4, 11, 1, 1] 132
cases of this type: 484
best so far: 0
type: [3, 11, 1, 1, 4] 132
cases of this type: 2816
best so far: 0
type: [3, 11, 1, 2, 2] 132
cases of this type: 1408
best so far: 0
type: [3, 11, 1, 4, 1] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 1, 2] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 2, 1] 132
cases of this type: 352
best so far: 0
type: [3, 11, 4, 1, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 1, 1, 2] 132
cases of this type: 352
best so far: 0
type: [3, 22, 1, 2, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 2, 1, 1] 132
cases of this type: 88
best so far: 0
type: [3, 44, 1, 1, 1] 132
cases of this type: 44
best so far: 0
type: [4, 1, 1, 1, 33] 132
cases of this type: 1185921
126